In [14]:
import torch

from tqdm.auto import tqdm

import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from sklearn.metrics import accuracy_score


In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [16]:
# torchvision.transforms, help with converting image to numbers
# torch.utils.data.Dataset, base dataset class
# torch.utils.data.DataLoader, creates python iterable over a dataset


In [17]:
train_data = datasets.FashionMNIST(
	root='datasets/',
	train=True,
	download=False,
	transform=transforms.ToTensor(),
	target_transform=None
)

test_data = datasets.FashionMNIST(
	root='datasets/',
	train=False,
	download=False,
	transform=transforms.ToTensor(),
	target_transform=None
)


In [18]:
BATCH_SIZE = 32

train_dl = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dl = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)


In [23]:
class FashionMNISTModel(nn.Module):
	def __init__(self, inputs, hiddens, outputs):
		super().__init__()

		self.layer_stack = nn.Sequential(
			nn.Flatten(),
			nn.Linear(in_features=inputs, out_features=hiddens),
			nn.ReLU(),
			nn.Linear(in_features=hiddens, out_features=hiddens),
			nn.ReLU(),
			nn.Linear(in_features=hiddens, out_features=outputs),
			nn.ReLU()
		)

	def forward(self, X: torch.Tensor) -> torch.Tensor:
		return self.layer_stack(X)

model = FashionMNISTModel(inputs=784, hiddens=16, outputs=10).to(device)


In [25]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)
epochs = 50

for epoch in range(1, 1+epochs):
	#print(f'----- {epoch = } -----')

	loss_per_epoch = 0
	model.train()
	for batch_number, (sample, label) in enumerate(train_dl):
		sample, label = sample.to(device), label.to(device)

		preds = model(sample)
		loss = loss_fn(preds, label)
		loss_per_epoch += loss

		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

	loss_per_epoch /= len(train_dl)

	model.eval()
	with torch.inference_mode():
		test_loss_per_epoch = 0
		accuracy_per_epoch = 0
		for (sample, label) in test_dl:
			sample, label = sample.to(device), label.to(device)

			test_preds = model(sample)
			test_loss = loss_fn(test_preds, label)
			test_loss_per_epoch += test_loss
			accuracy_per_epoch += accuracy_score(
				y_true=label.cpu(), y_pred=test_preds.cpu().argmax(dim=1)
			)

		test_loss_per_epoch /= len(test_dl)
		accuracy_per_epoch /= len(test_dl)

	if epoch%10 == 0 or epoch == 1:
		print(f'{epoch = }\t{loss_per_epoch = :.3f}\t{test_loss_per_epoch = :.3f}\t accuracy = {accuracy_per_epoch*100:.2f}%')


epoch = 1	loss_per_epoch = 0.277	test_loss_per_epoch = 0.416	 accuracy = 85.89%
epoch = 10	loss_per_epoch = 0.271	test_loss_per_epoch = 0.405	 accuracy = 86.34%
epoch = 20	loss_per_epoch = 0.268	test_loss_per_epoch = 0.420	 accuracy = 86.60%
epoch = 30	loss_per_epoch = 0.261	test_loss_per_epoch = 0.438	 accuracy = 86.16%
epoch = 40	loss_per_epoch = 0.256	test_loss_per_epoch = 0.447	 accuracy = 85.81%
epoch = 50	loss_per_epoch = 0.251	test_loss_per_epoch = 0.469	 accuracy = 86.07%


In [21]:
#! print(test_preds[49:52].argmax(dim=1))
#! print(y_test[49:52])

#plt.imshow(X_test[50].reshape((28,28)).cpu(), cmap='gray', vmin=0, vmax=1)
#print(test_data.classes[y_test[50]])
#print(test_data.classes[test_preds[50].argmax()])
